# Examples on using the client library for DWF

Note, this script was written using the **test server**, double check **which port the DB is running on**.

After importing the client module, create the client object specifying the DB url and the sender (author) for the requests. 

The following methods are available for records:

- `add_record`: takes a dictionary with the required input
- `add_batch`: reads in a file with a json list of several observations to add (not functional at the moment)
- `send_record`: creates the entry of the record added with add_record in the DB
- `update_record`: update an entry specified via maryID with the record added with add_record
- `get_record`: retieves the entry of the record based on RA, DEC, and distance from the DB


The following methods are available for comments associated with records (maryID):
- `add_post`: takes a dictionary with comments associated with a Mary ID
- `send_post`: sends the entry added with add_post to the DB, giving it a unique post ID, hence one maryID can have several post associated with it
- `get_post`: retieves entries of comments based on maryID from the DB



### Import the library and set up the Client

In [ ]:
#! /usr/bin/env python

import glob
from astropy.io import ascii
# import adacsdwf as dwf
from dwf_server import client as dwf
import requests


# setting up server side
url = '192.168.44.221:5000'
client = dwf.Client(url,"rebecca")

A quick check to see if the DB is up and running you can request the content from the front page.
This should return 'Hello World!' if all is well.

In [ ]:
r = requests.get(f"{url}/")
r.text

### Creat a dict with data to add to DB

In [ ]:
# creating test data
test1 = {
    "id":31903,
    "field": "8hr",
    "ccd": 18,
    "mary_run": 72,
    "date": 180607,
    "cand_num": 1,
    "mag": 21.35,
    "emag": 0.073,
    "mjd": 58276.02048322,
    "ra": 123.9024,
    "dec": -78.5336,
    "maryID": "8hr_mrt1_72_31903_180607",
    "sci_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180607_mrt1_72/8hr_180607_mrt1_72_stamp_ccd18_cand1_sci.fits",
    "sub_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180607_mrt1_72/8hr_180607_mrt1_72_stamp_ccd18_cand1_sub.fits",
    "temp_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180607_mrt1_72/8hr_180607_mrt1_72_stamp_ccd18_cand1_temp.fits"
}
        

test2 = {
    "id":62963,
    "field": "8hr",
    "ccd": 44,
    "mary_run": 84,
    "date": 180608,
    "cand_num": 1,
    "mag": 21.3197002,
    "emag": 0.0591000322,
    "mjd": 58277.03082293,
    "ra": 124.63092719,
    "dec": -79.10021861,
    "maryID": "8hr_mrt1_84_62963_180608",
    "sci_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_sci.fits",
    "sub_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_sub.fits",
    "temp_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_temp.fits"
}

test3 = {
    "id":31905,
    "field": "8hr",
    "ccd": 18,
    "mary_run": 72,
    "date": 180608,
    "cand_num": 3,
    "mag": 21.1539001,
    "emag": 0.0629000322,
    "mjd": 58276.02048322,
    "ra": 123.89938445,
    "dec": -78.506815259,
    "maryID": "8hr_mrt1_72_31905_180607",
    "sci_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180607_mrt1_72/8hr_180607_mrt1_72_stamp_ccd18_cand3_sci.fits",
    "sub_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180607_mrt1_72/8hr_180607_mrt1_72_stamp_ccd18_cand3_sub.fits",
    "temp_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180607_mrt1_72/8hr_180607_mrt1_72_stamp_ccd18_cand3_temp.fits"
}

### How to add entries to the DB

To add an entry to the DB, first add the record to the client which will check for data integrity, then send the record to the DB which will add a new line.

In [ ]:
# using client library function to add mary run
client.add_record(test1)
client.record

In [ ]:
# send to DB
client.send_record()

If you try to add an entry that does not fullfill basic data integrity you will get an error on the add_record stage

In [ ]:
test_bad = {
    "id":62963.,
    "field": "8hr",
    "ccd": 44,
    "mary_run": 84,
    "date": 180608,
    "cand_num": 1,
    "mag": 21.3197002,
    "emag": 0.0591000322,
    "mjd": 58277.03082293,
    "ra": 124.,
    "dec": -79.10021861,
    "maryID": "8hr_mrt1_84_62963_180608",
    "sci_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_sci.fits",
    "sub_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_sub.fits",
    "temp_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_temp.fits"
}

# using client library function to add mary run
client.add_record(test_bad)


You can us the `add_batch()` method to add a file containing an ascii table with one record per line and column names equal to the ones used in `.add_record()`.

This will check the validity of each row and send the content to the DB if checks are passed (same checks as for an individual entry).
If an entry does not pass the criteria the reason is added to a dataframe which is saved as the `batch` object and can be accessed with `client.batch`

In [ ]:
# using the library function to test and send a batch entry from file to the DB
client.add_batch(filename="test.ascii")

In [ ]:
# to check any errors that occured during the add_batch preocess
client.batch

### How to add a comment

Note, a comment has to be associated with a MaryID present in the DB, however, this is not tested until the comment is send to the DB! (It is on my to do list.) 

If no author is specified the identiy used to set up the client will be used.

In [ ]:
client.add_post(maryid="8hr_mrt1_84_62963_180608",body="yet another entry",author="rebecca")
client.post

In [ ]:
client.send_post()

**Note:** When a post entry is created in the DB it will also be associated with a datestamp.

This makes it possible to send the same post several times to the DB without an error.

### Retrieving posts and RA-DEC queries

In [ ]:
#! /usr/bin/env python

import glob
# import adacsdwf as dwf
from dwf_server import client as dwf
import requests

# setting up server side
url = 'http://127.0.0.1:5000'
client = dwf.Client(url,"rebecca")

In [ ]:
r = requests.get(f"{url}/")
r.text

**RA and Dec queries**

In [ ]:
r = client.get_record(ra=124.5, dec=-79, dist=0.2)

In [ ]:
r.json

**retieve comments**


In [ ]:
r = client.get_post(maryid="8hr_mrt1_84_62963_180608")

In [ ]:
r.json()

### How to update an entry

In [ ]:
#! /usr/bin/env python

import glob
# import adacsdwf as dwf
from dwf_server import client as dwf
import requests

# setting up server side
url = 'http://127.0.0.1:5000'
client = dwf.Client(url,"rebecca")

In [ ]:
test_update = {
    "id":62963,
    "field": "8hr",
    "ccd": 47,
    "mary_run": 84,
    "date": 180608,
    "cand_num": 1,
    "mag": 21.3197002,
    "emag": 0.0591000322,
    "mjd": 58277.03082293,
    "ra": 124.63092719,
    "dec": -79.10021861,
    "maryID": "8hr_mrt1_84_62963_180608",
    "sci_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_sci.fits",
    "sub_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_sub.fits",
    "temp_path": "/fred/oz100/pipes/DWF_PIPE/MARY_STAMP/8hr_180608_mrt1_84/8hr_180608_mrt1_84_stamp_ccd44_cand1_temp.fits"
}

In [ ]:
# using client library function to add mary run
client.add_record(test_update)

In [ ]:
r = client.update_record()

In [ ]:
r

It is possible to update a comment, however this can only be done via the requests libray and does currently not allow for author changes. See next section for an example

## How to query / post directly using the requests library

Note, if you are planning to use the request library I recommend using this for querying only.

If you want to use it for adding a DB entry, be aware that the client API offers several checks to make sure the data to be posted is valid.

Hence, I only show examples for sending info to the DB for posting comments. For querying I show both access to data and comments.

**Querying RA and Dec data**

In [ ]:
# Returning all entries in the DB
package = "web/run"
method = "all"

In [ ]:
r = requests.get(f"{url}/{package}/{method}")

In [ ]:
r.json()

In [ ]:
# Returning entries by RA, DEC and dist
package = "web/run"
run_payload = {"ra": 124.5, "dec": -79, "d": 0.2}

In [ ]:
r = requests.get(f"{url}/{package}", json = run_payload)

In [ ]:
r.json()

**posting, updating and retrieving comments**

In [ ]:
# POST request for comments
package = "web/post"
method = "create"


In [ ]:
post_payload = {"maryID":"8hr_mrt1_84_62963_180608", "body":"some comment about the mary object", "author":"rebecca"}

In [ ]:
r = requests.post(f"{url}/{package}/{method}", json=post_payload)


In [ ]:
# retieving comment posts using GET request
package = "web/post"
post_payload = {"maryID":"8hr_mrt1_84_62963_180608"}

In [ ]:
r = requests.get(f"{url}/{package}", json=post_payload)

In [ ]:
r.json()

In [ ]:
#  UPDATE request for comments
package = "web/post"
method = "update"
post_id = 1
post_payload = {"maryxID":"8hr_mrt1_84_62963_180608", "body":"updated test", "author":"rebecca"}

In [ ]:
r = requests.put(f"{url}/{package}/{post_id}/{method}", json=post_payload)

In [ ]:
r.json()